In [1]:
# カレントディレクトリをリポジトリ直下にするおまじない
import os
import tensorflow as tf 
print(tf.__version__)
print("hello world")
print(os.getcwd().split('\\')[-1] )
while os.getcwd().split('\\')[-1] != 'deep_dialog_tutorial': os.chdir('..')
print('current dir:', os.getcwd())


2.6.0
hello world
transformer
current dir: C:\Users\konolab\Desktop\Research\deep_dialog_tutorial


In [2]:
import os
print(os.getcwd())
import tensorflow as tf
from deepdialog.transformer.transformer import Transformer
from deepdialog.transformer.preprocess.batch_generator import BatchGenerator

C:\Users\konolab\Desktop\Research\deep_dialog_tutorial


# Create Data

In [3]:
data_path = 'data\\natsume.txt'

In [4]:
batch_generator = BatchGenerator()
batch_generator.load(data_path)

明治三十八年九月。


In [5]:
vocab_size = batch_generator.vocab_size

# Create Model

In [6]:
graph = tf.Graph()
with graph.as_default():
    transformer = Transformer(
        vocab_size=vocab_size,
        hopping_num=4,
        head_num=8,
        hidden_dim=512,
        dropout_rate=0.1,
        max_length=50,
    )
    transformer.build_graph()

C:\Users\konolab\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\engine\base_layer_v1.py:1684: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Instructions for updating:
Use `tf.cast` instead.


# Create Training Graph

In [7]:
save_dir = 'tmp/learning/transformer/'
log_dir = os.path.join(save_dir, 'log')
ckpt_path = os.path.join(save_dir, 'checkpoints/model.ckpt')

os.makedirs(log_dir, exist_ok=True)

In [8]:
with graph.as_default():
    global_step = tf.compat.v1.train.get_or_create_global_step()
    
    learning_rate = tf.compat.v1.placeholder(dtype=tf.float32, name='learning_rate')
    optimizer = tf.compat.v1.train.AdamOptimizer(
        learning_rate=learning_rate,
        beta2=0.98,
    )
    #TensorFlowには損失関数を最小限に抑えるために各変数をゆっくりと変更するオプティマイザが用意されている
    optimize_op = optimizer.minimize(transformer.loss, global_step=global_step)
# Tensor boardに出すためにデータをマージ
    summary_op = tf.compat.v1.summary.merge([
        tf.compat.v1.summary.scalar('train/loss', transformer.loss),
        tf.compat.v1.summary.scalar('train/acc', transformer.acc),
        tf.compat.v1.summary.scalar('train/learning_rate', learning_rate),
    ], name='train_summary')
    summary_writer = tf.compat.v1.summary.FileWriter(log_dir, graph)
    saver = tf.compat.v1.train.Saver()

# Train

In [9]:
max_step = 100000
batch_size = 128
max_learning_rate = 0.0001
warmup_step = 4000

In [10]:
def get_learning_rate(step: int) -> float:
    rate = min(step ** -0.5, step * warmup_step ** -1.5) / warmup_step ** -0.5
    return max_learning_rate * rate

In [11]:
with graph.as_default():
    sess = tf.compat.v1.Session()
    sess.run(tf.compat.v1.global_variables_initializer())
    step = 0

In [12]:
with graph.as_default():
    for batch in batch_generator.get_batch(batch_size=batch_size):
        feed = {
            **batch,
            learning_rate: get_learning_rate(step + 1),
        }
        _, loss, acc, step, summary = sess.run([optimize_op, transformer.loss, transformer.acc, global_step, summary_op], feed_dict=feed)
        summary_writer.add_summary(summary, step)
        
        if step % 100 == 0:
            print(f'{step}: loss: {loss},\t acc: {acc}')
            saver.save(sess, ckpt_path, global_step=step)
        if step == max_step:
            break

0: loss: 8.38064193725586,	 acc: 0.0
100: loss: 8.005585670471191,	 acc: 0.08455361425876617
200: loss: 7.597139358520508,	 acc: 0.095381960272789
300: loss: 7.288633346557617,	 acc: 0.12671375274658203
400: loss: 6.917326927185059,	 acc: 0.12110453844070435
500: loss: 6.578369617462158,	 acc: 0.1424790471792221
Instructions for updating:
Use standard file APIs to delete files with this prefix.
600: loss: 6.3707194328308105,	 acc: 0.14273612201213837
700: loss: 6.2278361320495605,	 acc: 0.14492753148078918
800: loss: 6.173699855804443,	 acc: 0.160411074757576
900: loss: 6.009583473205566,	 acc: 0.17045961320400238
1000: loss: 5.886408805847168,	 acc: 0.18925459682941437
1100: loss: 5.909269332885742,	 acc: 0.18838591873645782
1200: loss: 5.943912982940674,	 acc: 0.17588357627391815
1300: loss: 5.891147136688232,	 acc: 0.18591319024562836
1400: loss: 5.76690673828125,	 acc: 0.19243986904621124
1500: loss: 5.68817138671875,	 acc: 0.19531914591789246
1600: loss: 5.721025466918945,	 acc: 0

13900: loss: 3.6773340702056885,	 acc: 0.3391836881637573
14000: loss: 3.6346476078033447,	 acc: 0.32829269766807556
14100: loss: 3.688983201980591,	 acc: 0.3205893933773041
14200: loss: 3.7225842475891113,	 acc: 0.31488314270973206
14300: loss: 3.6767868995666504,	 acc: 0.32202696800231934
14400: loss: 3.6198010444641113,	 acc: 0.3294307589530945
14500: loss: 3.6709582805633545,	 acc: 0.32439228892326355
14600: loss: 3.603006601333618,	 acc: 0.31957894563674927
14700: loss: 3.647874593734741,	 acc: 0.31879332661628723
14800: loss: 3.71213698387146,	 acc: 0.3197721838951111
14900: loss: 3.6779017448425293,	 acc: 0.31466439366340637
15000: loss: 3.590003728866577,	 acc: 0.32463642954826355
15100: loss: 3.6037368774414062,	 acc: 0.33112582564353943
15200: loss: 3.605116844177246,	 acc: 0.32281261682510376
15300: loss: 3.5642473697662354,	 acc: 0.33985838294029236
15400: loss: 3.5256729125976562,	 acc: 0.3532294034957886
15500: loss: 3.6113011837005615,	 acc: 0.3202110528945923
15600: los

28100: loss: 2.9800026416778564,	 acc: 0.4011434018611908
28200: loss: 2.8576133251190186,	 acc: 0.4293603301048279
28300: loss: 2.9630942344665527,	 acc: 0.4103635549545288
28400: loss: 2.8744771480560303,	 acc: 0.4199395775794983
28500: loss: 2.9073195457458496,	 acc: 0.41908714175224304
28600: loss: 2.9244439601898193,	 acc: 0.4074544608592987
28700: loss: 2.9889144897460938,	 acc: 0.3922574520111084
28800: loss: 3.0271265506744385,	 acc: 0.3842364549636841
28900: loss: 3.041658639907837,	 acc: 0.40114378929138184
29000: loss: 2.8987042903900146,	 acc: 0.4167032837867737
29100: loss: 2.9315223693847656,	 acc: 0.39761093258857727
29200: loss: 2.9142751693725586,	 acc: 0.40638670325279236
29300: loss: 2.850557327270508,	 acc: 0.4111498296260834
29400: loss: 2.879891872406006,	 acc: 0.4025586247444153
29500: loss: 3.0047595500946045,	 acc: 0.3866778314113617
29600: loss: 2.930553913116455,	 acc: 0.41397204995155334
29700: loss: 3.0085511207580566,	 acc: 0.38837921619415283
29800: loss:

42300: loss: 2.5274930000305176,	 acc: 0.45555099844932556
42400: loss: 2.4898674488067627,	 acc: 0.4689413905143738
42500: loss: 2.573925495147705,	 acc: 0.4508414566516876
42600: loss: 2.485491991043091,	 acc: 0.4816947877407074
42700: loss: 2.4980392456054688,	 acc: 0.46958741545677185
42800: loss: 2.500871181488037,	 acc: 0.46559232473373413
42900: loss: 2.523160219192505,	 acc: 0.47550562024116516
43000: loss: 2.640096426010132,	 acc: 0.43867743015289307
43100: loss: 2.4571382999420166,	 acc: 0.48493486642837524
43200: loss: 2.49731707572937,	 acc: 0.4758928716182709
43300: loss: 2.4554085731506348,	 acc: 0.46663719415664673
43400: loss: 2.4119925498962402,	 acc: 0.4916520118713379
43500: loss: 2.4908857345581055,	 acc: 0.46380579471588135
43600: loss: 2.5628607273101807,	 acc: 0.4580419659614563
43700: loss: 2.561756134033203,	 acc: 0.45478036999702454
43800: loss: 2.5388731956481934,	 acc: 0.4602917432785034
43900: loss: 2.4747321605682373,	 acc: 0.4673311114311218
44000: loss: 

56500: loss: 2.138678550720215,	 acc: 0.5257819294929504
56600: loss: 2.187636613845825,	 acc: 0.5135593414306641
56700: loss: 2.2242980003356934,	 acc: 0.5119296908378601
56800: loss: 2.1412391662597656,	 acc: 0.5214669108390808
56900: loss: 2.281093120574951,	 acc: 0.5050792098045349
57000: loss: 2.2103474140167236,	 acc: 0.5217959880828857
57100: loss: 2.174997091293335,	 acc: 0.522020697593689
57200: loss: 2.160001277923584,	 acc: 0.5184534192085266
57300: loss: 2.1374595165252686,	 acc: 0.5315024256706238
57400: loss: 2.21144437789917,	 acc: 0.518305242061615
57500: loss: 2.107307195663452,	 acc: 0.5365955829620361
57600: loss: 2.1648213863372803,	 acc: 0.5222429633140564
57700: loss: 2.2817118167877197,	 acc: 0.5016474723815918
57800: loss: 2.1610937118530273,	 acc: 0.5243902206420898
57900: loss: 2.2652435302734375,	 acc: 0.5004504323005676
58000: loss: 2.1110165119171143,	 acc: 0.5303802490234375
58100: loss: 2.110727548599243,	 acc: 0.538394033908844
58200: loss: 2.13158631324

70800: loss: 1.9063084125518799,	 acc: 0.5691699385643005
70900: loss: 1.9440536499023438,	 acc: 0.5584970116615295
71000: loss: 1.937070608139038,	 acc: 0.5600951910018921
71100: loss: 1.909623146057129,	 acc: 0.5670378804206848
71200: loss: 2.023016929626465,	 acc: 0.5482401847839355
71300: loss: 1.8722662925720215,	 acc: 0.5660211443901062
71400: loss: 1.889251708984375,	 acc: 0.5831485390663147
71500: loss: 1.9140040874481201,	 acc: 0.5559380650520325
71600: loss: 1.8795925378799438,	 acc: 0.5686873197555542
71700: loss: 1.9444867372512817,	 acc: 0.558590292930603
71800: loss: 1.8953036069869995,	 acc: 0.5675547122955322
71900: loss: 1.9763180017471313,	 acc: 0.5484727621078491
72000: loss: 1.9391448497772217,	 acc: 0.5625978112220764
72100: loss: 1.8749867677688599,	 acc: 0.5761865377426147
72200: loss: 1.9272969961166382,	 acc: 0.5635899901390076
72300: loss: 1.8800153732299805,	 acc: 0.5769230723381042
72400: loss: 1.8961070775985718,	 acc: 0.5689731240272522
72500: loss: 1.8963

85100: loss: 1.7470825910568237,	 acc: 0.6076003313064575
85200: loss: 1.7248831987380981,	 acc: 0.6028896570205688
85300: loss: 1.6584270000457764,	 acc: 0.6199073791503906
85400: loss: 1.656914472579956,	 acc: 0.6225983500480652
85500: loss: 1.6454856395721436,	 acc: 0.6240074634552002
85600: loss: 1.7250523567199707,	 acc: 0.6112047433853149
85700: loss: 1.651369571685791,	 acc: 0.6241640448570251
85800: loss: 1.6661303043365479,	 acc: 0.6038239002227783
85900: loss: 1.6387468576431274,	 acc: 0.6230769157409668
86000: loss: 1.765716314315796,	 acc: 0.5967885851860046
86100: loss: 1.7049496173858643,	 acc: 0.6142383813858032
86200: loss: 1.5764057636260986,	 acc: 0.6459197998046875
86300: loss: 1.636006236076355,	 acc: 0.6260199546813965
86400: loss: 1.651803731918335,	 acc: 0.6239199638366699
86500: loss: 1.7720582485198975,	 acc: 0.6000000238418579
86600: loss: 1.6819418668746948,	 acc: 0.5972850918769836
86700: loss: 1.6755688190460205,	 acc: 0.6103951930999756
86800: loss: 1.7057

99400: loss: 1.5242462158203125,	 acc: 0.6467273235321045
99500: loss: 1.5048803091049194,	 acc: 0.6474589705467224
99600: loss: 1.559770941734314,	 acc: 0.6353591084480286
99700: loss: 1.5757719278335571,	 acc: 0.6289808750152588
99800: loss: 1.535814642906189,	 acc: 0.6384720206260681
99900: loss: 1.5350067615509033,	 acc: 0.641385018825531
100000: loss: 1.5560190677642822,	 acc: 0.6207455396652222
